# FAISS DB

Facebook AI Similarity Search

In [ ]:
# 학원 대여 노트북은 gpu로 계산 안하고 cpu로만 계산해서
#!pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   --------------------------- ------------ 12.6/18.2 MB 65.9 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 60.3 MB/s  0:00:00


### 영화 추천 시스템

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
movies = [
    {"id": 0, "title": "인셉션", "description": "꿈속에서 다른 사람의 생각을 조작하는 스토리"},
    {"id": 1, "title": "어벤져스: 엔드게임", "description": "어벤져스가 타노스를 물리치기 위한 마지막 전투를 벌이는 이야기"},
    {"id": 2, "title": "기생충", "description": "부유한 가정과 가난한 가정의 충격적인 만남을 그린 사회적 드라마"},
    {"id": 3, "title": "해리 포터와 마법사의 돌", "description": "마법의 세계에 입학한 소년 해리 포터의 첫 번째 모험"},
    {"id": 4, "title": "반지의 제왕: 왕의 귀환", "description": "반지를 파괴하기 위한 최후의 전투가 벌어지는 이야기"},
    {"id": 5, "title": "스타워즈: 제국의 역습", "description": "제국의 추격을 피해 도망치는 반란군들의 이야기"},
    {"id": 6, "title": "타이타닉", "description": "침몰하는 타이타닉 호에서의 사랑과 비극적인 운명을 그린 이야기"},
    {"id": 7, "title": "쇼생크 탈출", "description": "불법적인 이유로 감옥에 갇힌 남자가 탈출을 계획하는 이야기"},
    {"id": 8, "title": "인터스텔라", "description": "우주를 넘어서 인류의 생존을 위한 탐험을 그린 이야기"},
    {"id": 9, "title": "블랙 팬서", "description": "와칸다 왕국의 새로운 왕이 되어가는 이야기를 그린 슈퍼히어로 영화"},
    {"id": 10, "title": "어드벤처 타임", "description": "모험과 상상력으로 가득한 세계에서 주인공들이 펼치는 이야기"},
    {"id": 11, "title": "혹성 탈출", "description": "인간과 유인원의 관계를 그린 고전적인 SF 영화"},
    {"id": 12, "title": "죽은 시인의 사회", "description": "고등학교 교사와 학생들이 펼치는 삶의 의미를 찾는 이야기"},
    {"id": 13, "title": "올드보이", "description": "13년간 감금된 남자가 복수를 위해 싸우는 미스터리 영화"},
    {"id": 14, "title": "아이언맨", "description": "억만장자 천재 발명가가 슈퍼히어로로 변신하는 이야기"},
    {"id": 15, "title": "포레스트 검프", "description": "특별한 능력 없이 살아온 한 남자의 삶을 그린 감동적인 이야기"},
    {"id": 16, "title": "카카오", "description": "기업 카카오의 창업과 성공 스토리를 그린 영화"},
    {"id": 17, "title": "디지털 포레스트", "description": "인터넷과 디지털 기술로 뒤얽힌 사회와 인간의 관계를 그린 영화"},
    {"id": 18, "title": "미션 임파서블: 고스트 프로토콜", "description": "특수 요원들이 전 세계에서 벌어지는 테러를 막기 위해 고군분투하는 이야기"},
    {"id": 19, "title": "어둠의 경로", "description": "밤마다 악몽을 겪는 주인공이 자신만의 진실을 찾아가는 스릴러 영화"}
]

In [5]:
import numpy as np

descriptions = [movie['description'] for movie in movies]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype='float32')

In [7]:
import faiss
 
dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim) # chromadb에서 collection이랑 같은 거 == Index
idx.add(desc_embed)

In [10]:
query_text = '탈출하는 이야기'
query_embed = np.array([model.encode(query_text)], dtype='float32')

In [11]:
top_n = 5
distance, indices = idx.search(query_embed, top_n)

print(distance)
print(indices)

[[0.8666453 0.8801925 0.9008679 0.9916291 1.0403186]]
[[ 0 13  4  3 18]]


In [16]:
for i in range(top_n):
    movie_id = indices[0][i]
    print(f"{i+1}번째 추천: {movies[movie_id]['title']} (유사도 거리: {distance[0][i]:.2f})")

1번째 추천: 인셉션 (유사도 거리: 0.87)
2번째 추천: 올드보이 (유사도 거리: 0.88)
3번째 추천: 반지의 제왕: 왕의 귀환 (유사도 거리: 0.90)
4번째 추천: 해리 포터와 마법사의 돌 (유사도 거리: 0.99)
5번째 추천: 미션 임파서블: 고스트 프로토콜 (유사도 거리: 1.04)


### 사용자 맞춤 뉴스 추천

In [20]:
# 키워드 받아서 네이버 뉴스 제목 크롤링 (상위 10건만) 
# -> FAISS 활용 
# -> 특정 키워드와 관련된 뉴스 2개만 출력

# 1. 제목 클롤링
import requests
from bs4 import BeautifulSoup

keyword = input("뉴스 검색 키워드: ")
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={keyword}"
response = requests.get(url) # response 객체로 받음
html = response.text
bs = BeautifulSoup(html, 'html.parser') # html parser를 통해서 html text를 파싱

news_title_tags = bs.select('a.Pf2RblZWtkibrjn1Pq42.EnZfm_hC1iOYERXp71Cm > span')
titles = [news_title_tag.text for news_title_tag in news_title_tags]

In [25]:
print(titles)

['블랙핑크 제니, 월세 6000만 빌딩으로 ‘통큰’ 이사…제니는 제니다', '블랙핑크 제니, 450억 한남동 신축 건물로 사무실 이전', "샥즈, 클립 이어폰 '오픈닷 원 핑크' 출시", '야노시호, 블랙핑크·BTS RM과 초특급 만남 "반가워요"', '블랙핑크, BTS? 2025 파라다이스 아트 나잇에 간 셀럽들!', '블랙핑크 리사, 아이브 안유진 등 글로벌 스타들, 불닭에 푹 빠졌다', "블랙핑크 리사, 속옷 보이려 바지 접어…토마토 가방은 '충격적'", '[문화연예 플러스] 블랙핑크 로제 "유튜브 수익만 110억 원"', '“유튜브 年수익만 110억”…블랙핑크 로제, 셀럽 유튜버 1위', '야노 시호, 블랙핑크·BTS RM과 만남 인증…보랏빛 드레스로 뽐낸 우아함']


In [22]:
# 2. FAISS 하기
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer('all-MiniLM-L6-v2')
titles_embed = np.array([model.encode(title) for title in titles], dtype='float32')

dim = titles_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(titles_embed)

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
# 3. 특정 키워드와 관련된 뉴스 2개만 출력
query_text = 'BTS'
query_embed = np.array([model.encode(query_text)], dtype='float32')

top_n = 2
distance, indices = idx.search(query_embed, top_n)

for i in range(top_n):
    print(f"{i+1}번째 추천: {titles[indices[0][i]]} (유사도 거리: {distance[0][i]:.2f})")

1번째 추천: 블랙핑크, BTS? 2025 파라다이스 아트 나잇에 간 셀럽들! (유사도 거리: 0.96)
2번째 추천: 야노 시호, 블랙핑크·BTS RM과 만남 인증…보랏빛 드레스로 뽐낸 우아함 (유사도 거리: 1.00)
